In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, LassoCV, LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import (GridSearchCV, cross_val_score, train_test_split)
from sklearn.preprocessing import StandardScaler
data = pd.read_csv('winequality-white.csv')
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


In [3]:
y = data['quality']
X = data.drop(['quality'], axis=1)
X_train, X_holdout, y_train, y_holdout = train_test_split(X,y,train_size=0.7, random_state=17)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)
print("Mean squared error(train):",mean_squared_error(y_train,linreg.predict(X_train_scaled)))
print("Mean squared error(test):",mean_squared_error(y_holdout,linreg.predict(X_holdout_scaled)))

Mean squared error(train): 0.5580606489803572
Mean squared error(test): 0.5842473102404546


In [18]:
linreg_coef = pd.DataFrame(linreg.coef_,data.columns[:-1],columns=["coef"])
linreg_coef.sort_values(by="coef",ascending=False)

,coef
residual sugar,0.538164
pH,0.150036
alcohol,0.129533
fixed acidity,0.097822
sulphates,0.062053
free sulfur dioxide,0.042180
total sulfur dioxide,0.014304
chlorides,0.008127
citric acid,-0.000183
volatile acidity,-0.192260


In [5]:
lasso1 = Lasso(alpha=0.01, random_state=17)
lasso1.fit(X_train_scaled,y_train)
lasso1_coef = pd.DataFrame(lasso1.coef_,data.columns[:-1],columns=["coef"])
lasso1_coef.sort_values(by="coef",ascending=False)

,coef
alcohol,0.322425
residual sugar,0.256363
pH,0.067277
free sulfur dioxide,0.043088
sulphates,0.029722
fixed acidity,-0.000000
citric acid,-0.000000
total sulfur dioxide,-0.000000
chlorides,-0.002747
volatile acidity,-0.188479


In [6]:
alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(cv=5,random_state=17)
lasso_cv.fit(X_train_scaled,y_train)

LassoCV(cv=5, random_state=17)

In [7]:
lasso_cv.alpha_

0.0003898518324955784

In [8]:
lasso_cv_coef = pd.DataFrame(lasso_cv.coef_,data.columns[:-1],columns=["coef"])
lasso_cv_coef.sort_values(by="coef",ascending=False)

,coef
residual sugar,0.522635
pH,0.145230
alcohol,0.139980
fixed acidity,0.091608
sulphates,0.060524
free sulfur dioxide,0.042894
total sulfur dioxide,0.012472
chlorides,0.006492
citric acid,-0.000000
volatile acidity,-0.191982


In [9]:
print(mean_squared_error(y_train,lasso_cv.predict(X_train_scaled)))
print(mean_squared_error(y_holdout,lasso_cv.predict(X_holdout_scaled)))

0.5580783463961341
0.582952011239391


In [10]:
forest = RandomForestRegressor(random_state=17)
forest.fit(X_train_scaled,y_train)
print(mean_squared_error(y_train,forest.predict(X_train_scaled)))
print(np.mean(cross_val_score(forest,X_train,y_train,cv=5,scoring="neg_mean_squared_error")))
print(mean_squared_error(y_holdout,forest.predict(X_holdout_scaled)))


0.05261155192532089
-0.41433128807644015
0.37163775510204083


In [ ]:
forest_params = {'max_depth': list(range(10, 25)), 
                 'min_samples_leaf': list(range(1, 8)),
                 'max_features': list(range(6,12))}
locally_best_forest = GridSearchCV(forest,forest_params,cv=5)
locally_best_forest.fit(X_train,y_train)

In [ ]:
locally_best_forest.best_params_, locally_best_forest.best_score_

In [13]:
forest2 = RandomForestRegressor(max_depth=19,max_features=7,min_samples_leaf=1)
forest2.fit(X_train_scaled,y_train)
print(mean_squared_error(y_train,forest2.predict(X_train_scaled)))
print(np.mean(cross_val_score(forest2,X_train_scaled,y_train,cv=5,scoring="neg_mean_squared_error")))
print(mean_squared_error(y_holdout,forest2.predict(X_holdout_scaled)))

0.059937124853392615
-0.4030115718994168
0.3737107335961661


In [15]:
rf_importance = pd.DataFrame(forest2.feature_importances_,data.columns[:-1],columns=["importance"])
rf_importance.sort_values(by="importance",ascending=False)

,importance
alcohol,0.206634
volatile acidity,0.118593
free sulfur dioxide,0.110672
density,0.095925
residual sugar,0.073879
total sulfur dioxide,0.071898
pH,0.071276
chlorides,0.067874
fixed acidity,0.063248
citric acid,0.062017
